In [1]:
import pandas as pd
import numpy as np

In [2]:
submissions = ['resnet', 'keras_baseline']

In [5]:
submissions_dfs = []
for submission in submissions:
    submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + submission + "/" + submission + ".csv")
    submission_df = submission_df.sort('id')
    submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  after removing the cwd from sys.path.


In [2]:
submissions_dfs = []
paths = ['data_aug/flip_full.csv', 'ensamble/ensamble_0211_full.csv','pseudo_labeling/pseudo_full.csv', 'data_aug/black_box_full.csv','data_aug/noise_box_full.csv','data_aug/noise_mix_full.csv']
#paths = ['ensamble/stacking_0612_full.csv','data_aug/noise_mix_full.csv']
for path in paths:
    submission_df = pd.read_csv("/mnt/extDisk/courses/data/statoil/data/" + path)
    submission_df = submission_df.sort('id')
    submissions_dfs.append(submission_df)

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  


In [20]:
ensamble = submissions_dfs[0]
ensamble['is_iceberg'] = np.mean([submissions_dfs[0].is_iceberg,submissions_dfs[1].is_iceberg],axis=0)

In [21]:
ensamble.head()

,id,is_iceberg
0,000c65d7,0.901355
1,001680af,0.097190
2,00193999,0.986390
3,00247e48,0.724458
4,00503faa,0.019494


In [24]:
ensamble.to_csv("/mnt/extDisk/courses/data/statoil/data/ensamble/ensamble_0612_full.csv", index=False)

In [18]:
def mean_distance(df1,df2):
    print np.mean(np.absolute(df1['is_iceberg'] - df2['is_iceberg']))

In [19]:
mean_distance(submissions_dfs[0],submissions_dfs[1])

0.386040208166


In [3]:
for i in range(len(submissions_dfs)):
    submissions_dfs[i] = submissions_dfs[i].rename(columns={'is_iceberg':'is_iceberg_' +str(i)})

concat_sub = submissions_dfs[0]

for i in range(1,len(submissions_dfs)):
    concat_sub = concat_sub.merge(submissions_dfs[i],on='id')

#concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5
0,000c65d7,0.883821,0.948007,0.951305,0.536584,0.883821,0.907199
1,001680af,0.206122,0.397580,0.087387,0.019671,0.206122,0.063468
2,00193999,0.969658,0.965124,0.992847,0.932687,0.969658,0.984238
3,00247e48,0.626862,0.974896,0.999299,0.558587,0.626862,0.734220
4,00503faa,0.077954,0.240479,0.019667,0.000838,0.077954,0.000007


In [4]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:].median(axis=1)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median
0,000c65d7,0.883821,0.948007,0.951305,0.536584,0.883821,0.907199,0.951305,0.536584,0.824828,0.883821
1,001680af,0.206122,0.397580,0.087387,0.019671,0.206122,0.063468,0.397580,0.019671,0.174700,0.174700
2,00193999,0.969658,0.965124,0.992847,0.932687,0.969658,0.984238,0.992847,0.932687,0.967468,0.969658
3,00247e48,0.626862,0.974896,0.999299,0.558587,0.626862,0.734220,0.999299,0.558587,0.759827,0.734220
4,00503faa,0.077954,0.240479,0.019667,0.000838,0.077954,0.000007,0.240479,0.000007,0.082173,0.077954


In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [6]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:7] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:7] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub['is_iceberg'].head()

0    0.883821
1    0.174700
2    0.992847
3    0.734220
4    0.077954
Name: is_iceberg, dtype: float64

In [7]:
concat_sub[['id','is_iceberg']].head()

,id,is_iceberg
0,000c65d7,0.883821
1,001680af,0.174700
2,00193999,0.992847
3,00247e48,0.734220
4,00503faa,0.077954


In [8]:
concat_sub[['id','is_iceberg']].to_csv("/mnt/extDisk/courses/data/statoil/data/ensamble/stacking_1212_full.csv", index=False)

In [13]:
submissions_dfs[0].head()

,id,is_iceberg
0,000c65d7,0.895510
1,001680af,0.130912
2,00193999,0.988543
3,00247e48,0.714697
4,00503faa,0.038981


In [14]:
submissions_dfs[1].head()

,id,is_iceberg
5442,000c65d7,0.907199
37,001680af,0.063468
697,00193999,0.984238
1152,00247e48,0.734220
3033,00503faa,0.000007


In [23]:
(0.038981 +0.000007)/2

0.019494